<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/Emotionline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emitionline 데이터셋

In [1]:
! tar -xvzf /content/drive/My\ Drive/Datasets/Emotionlines/EmotionLines_friends_annotation.tar.gz

EmotionLines/Friends/friends_train.json
EmotionLines/Friends/friends_dev.json
EmotionLines/Friends/friends_test.json
EmotionLines/README


In [2]:
import pandas as pd
import json

In [3]:
with open('/content/EmotionLines/Friends/friends_train.json', encoding='utf-8', mode='r') as f:
  array = json.load(f)

data =[]
for i in range(len(array)):
  data += array[i]
print(f"Total length of data : {len(data)}")
df = pd.DataFrame(data)

Total length of data : 10561


In [4]:
df

,speaker,utterance,emotion,annotation
0,Chandler,also I was the point person on my companys tr...,neutral,4100000
1,The Interviewer,You mustve had your hands full.,neutral,5000000
2,Chandler,That I did. That I did.,neutral,5000000
3,The Interviewer,So lets talk a little bit about your duties.,neutral,5000000
4,Chandler,My duties? All right.,surprise,2000030
...,...,...,...,...
10556,Chandler,You or me?,neutral,3000011
10557,Ross,"I got it. Uh, Joey, women don't have Adam's ap...",non-neutral,2100011
10558,Joey,"You guys are messing with me, right?",surprise,0000050
10559,All,Yeah.,neutral,4000010


In [5]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [6]:
# change the value to dtype: Category
df['emotion'] = pd.Categorical(df['emotion'])

# cat(egory).codes print the code of category
df['emotion'] = df.emotion.cat.codes

In [7]:
df.emotion

0        4
1        4
2        4
3        4
4        7
        ..
10556    4
10557    5
10558    7
10559    4
10560    5
Name: emotion, Length: 10561, dtype: int8

In [8]:
df['emotion'].value_counts()

4    4752
5    2017
3    1283
7    1220
0     513
6     351
1     240
2     185
Name: emotion, dtype: int64

In [9]:
for i, utter in enumerate(df.utterance):
  if i < 5:
    print(utter)
  else:
    break

also I was the point person on my companys transition from the KL-5 to GR-6 system.
You mustve had your hands full.
That I did. That I did.
So lets talk a little bit about your duties.
My duties?  All right.


# Tensorflow

make the dataset to tensor

In [10]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

In [11]:
target = df.pop('utterance')

In [12]:
target.values

array(['also I was the point person on my company\x92s transition from the KL-5 to GR-6 system.',
       'You must\x92ve had your hands full.', 'That I did. That I did.',
       ..., 'You guys are messing with me, right?', 'Yeah.',
       'That was a good one. For a second there, I was like, "whoa."'],
      dtype=object)

In [13]:
df.values

array([['Chandler', 4, '4100000'],
       ['The Interviewer', 4, '5000000'],
       ['Chandler', 4, '5000000'],
       ...,
       ['Joey', 7, '0000050'],
       ['All', 4, '4000010'],
       ['Joey', 5, '1200020']], dtype=object)

In [14]:
dataset = tf.data.Dataset.from_tensor_slices(target)

In [15]:
for t in dataset.take(5):
  print(t)

tf.Tensor(b'also I was the point person on my company\xc2\x92s transition from the KL-5 to GR-6 system.', shape=(), dtype=string)
tf.Tensor(b'You must\xc2\x92ve had your hands full.', shape=(), dtype=string)
tf.Tensor(b'That I did. That I did.', shape=(), dtype=string)
tf.Tensor(b'So let\xc2\x92s talk a little bit about your duties.', shape=(), dtype=string)
tf.Tensor(b'My duties?  All right.', shape=(), dtype=string)


In [16]:
train_data = dataset.shuffle(len(df))
for t in train_data.take(5):
  print(t)

tf.Tensor(b'I think somebody\xc2\x92s got a little crush on Casey.', shape=(), dtype=string)
tf.Tensor(b'How was your first day?', shape=(), dtype=string)
tf.Tensor(b'Hey, when I\xc2\x92m in a play and you\xc2\x92re in the audience, I don\xc2\x92t talk to you, right?', shape=(), dtype=string)
tf.Tensor(b'Hey, Mon.', shape=(), dtype=string)
tf.Tensor(b'"Hi Joey, it\'s Jane Rogers can\'t wait for your party tonight"', shape=(), dtype=string)


# Tokenizer

In [100]:
tokenizer = K.preprocessing.text.Tokenizer(num_words = 4000, char_level=False)

In [101]:
tokenizer.fit_on_texts(target.values)

In [118]:
x_in = tokenizer.texts_to_sequences(target)
# print(x_in)

In [109]:
print(tokenizer.num_words)

4000


In [120]:
x_in = np.array(x_in)
print(x_in)

[list([410, 1, 35, 3, 762, 441, 29, 20, 2885, 2886, 127, 3, 2887, 1157, 4, 2888, 905, 906])
 list([2, 907, 102, 40, 709, 827]) list([9, 1, 73, 9, 1, 73]) ...
 list([2, 86, 32, 2874, 36, 14, 34]) list([17])
 list([9, 35, 5, 75, 62, 33, 5, 265, 57, 1, 35, 42, 177])]


# Embedding

In [106]:
from tensorflow.keras.layers import Embedding, InputLayer, Flatten

Model Create

In [111]:
model = K.Sequential(
    Embedding(input_dim = 4000, output_dim=100, )
)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         400000    
Total params: 400,000
Trainable params: 400,000
Non-trainable params: 0
_________________________________________________________________


In [112]:
model.compile('rmsprop', 'mse')

In [122]:
output = model.predict(x_in[0], batch_size=1)

In [123]:
print(output.shape)

(18, 1, 100)
